In [9]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2024-07-18  230.279999  230.440002  222.270004  224.179993  223.920807   
       2024-07-19  224.820007  226.800003  223.279999  224.309998  224.050659   
       2024-07-22  227.009995  227.779999  223.089996  223.960007  223.701080   
       2024-07-23  224.369995  226.940002  222.679993  225.009995  224.749847   
       2024-07-24  224.000000  224.800003  217.130005  218.539993  218.287323   

                     Volume  
Ticker Date                  
AAPL   2024-07-18  66034600  
       2024-07-19  49151500  
       2024-07-22  48201800  
       2024-07-23  39960300  
       2024-07-24  61777600  


In [10]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2024-07-18  230.279999  230.440002  222.270004  224.179993   
1   AAPL 2024-07-19  224.820007  226.800003  223.279999  224.309998   
2   AAPL 2024-07-22  227.009995  227.779999  223.089996  223.960007   
3   AAPL 2024-07-23  224.369995  226.940002  222.679993  225.009995   
4   AAPL 2024-07-24  224.000000  224.800003  217.130005  218.539993   

    Adj Close    Volume  
0  223.920807  66034600  
1  224.050659  49151500  
2  223.701080  48201800  
3  224.749847  39960300  
4  218.287323  61777600  


In [11]:
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [12]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [13]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
60  227.205000  225.921999
61  227.035001  226.670999
62  227.799001  227.524000
63  228.299001  228.078500
64  228.947000  228.242500

[65 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
195         NaN         NaN
196         NaN         NaN
197         NaN         NaN
198         NaN         NaN
199         NaN         NaN
..          ...         ...
255  165.953999  164.316000
256  165.870000  164.684000
257  165.717999  165.015000
258  165.661000  165.311501
259  165.390999  165.375000

[65 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  417.459000  424.99

In [14]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [15]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [35]:
# Check the contents of the merged DataFrame
print(df_corr.head())
print(df_corr.info())


Empty DataFrame
Columns: [Date, AAPL, MSFT]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    0 non-null      object
 1   AAPL    0 non-null      int64 
 2   MSFT    0 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 132.0+ bytes
None


In [36]:
# Check the data types of the columns
print(df_corr.dtypes)


Date    object
AAPL     int64
MSFT     int64
dtype: object


In [37]:
# Convert columns to numeric, if necessary
df_corr['AAPL'] = pd.to_numeric(df_corr['AAPL'], errors='coerce')
df_corr['MSFT'] = pd.to_numeric(df_corr['MSFT'], errors='coerce')

# Drop any rows with NaN values after conversion
df_corr.dropna(inplace=True)


In [40]:
# Print the first few rows of the original DataFrame
print(df.head())
print(df.columns)  # Check the column names


        Date Ticker  Close
0 2024-01-01   AAPL    150
1 2024-01-02   AAPL    155
2 2024-01-03   MSFT    250
3 2024-01-04   MSFT    255
4 2024-01-05   AAPL    160
Index(['Date', 'Ticker', 'Close'], dtype='object')


In [45]:
# Merge with outer join to include all dates
df_corr = pd.merge(apple, microsoft, on='Date', how='outer')


In [46]:
import plotly.express as px

# Simple line plot for Apple stock prices
fig = px.line(apple, x='Date', y='AAPL', title='Apple Stock Prices Over Time')
fig.show()

# Simple line plot for Microsoft stock prices
fig = px.line(microsoft, x='Date', y='MSFT', title='Microsoft Stock Prices Over Time')
fig.show()


In [50]:
import pandas as pd
import plotly.express as px

# Sample data creation (replace this with your actual data loading code)
data = {
    'Date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05'],
    'Ticker': ['AAPL', 'AAPL', 'MSFT', 'MSFT', 'AAPL'],
    'Close': [150, 155, 250, 255, 160]
}
df = pd.DataFrame(data)

# Ensure 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Print the original DataFrame and its columns
print("Original DataFrame:")
print(df)
print("Columns:", df.columns)

# Filter for Apple and Microsoft
apple = df[df['Ticker'] == 'AAPL'][['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df[df['Ticker'] == 'MSFT'][['Date', 'Close']].rename(columns={'Close': 'MSFT'})

# Print filtered DataFrames to confirm creation
print("Apple DataFrame:")
print(apple)
print("Microsoft DataFrame:")
print(microsoft)

# Set the date as the index
# Check if the 'Date' column exists before setting the index
if 'Date' in apple.columns and 'Date' in microsoft.columns:
    apple.set_index('Date', inplace=True)
    microsoft.set_index('Date', inplace=True)
else:
    print("Date column is missing in one of the DataFrames.")

# Resample to daily frequency and forward fill missing values
apple_resampled = apple.resample('D').ffill().reset_index()
microsoft_resampled = microsoft.resample('D').ffill().reset_index()

# Now merge the resampled data
df_corr = pd.merge(apple_resampled, microsoft_resampled, on='Date', how='outer')

# Check merged DataFrame
print("Merged DataFrame shape:", df_corr.shape)
print("Merged Data:\n", df_corr)

# Convert to numeric if needed
df_corr['AAPL'] = pd.to_numeric(df_corr['AAPL'], errors='coerce')
df_corr['MSFT'] = pd.to_numeric(df_corr['MSFT'], errors='coerce')

# Drop NaN values
df_corr.dropna(inplace=True)

# Check if data exists before plotting
if not df_corr.empty:
    # Simple line plot for both Apple and Microsoft stock prices
    fig = px.line(df_corr, x='Date', y=['AAPL', 'MSFT'], title='Stock Prices of Apple and Microsoft Over Time')
    fig.show()
else:
    print("Merged DataFrame is empty.")


Original DataFrame:
        Date Ticker  Close
0 2024-01-01   AAPL    150
1 2024-01-02   AAPL    155
2 2024-01-03   MSFT    250
3 2024-01-04   MSFT    255
4 2024-01-05   AAPL    160
Columns: Index(['Date', 'Ticker', 'Close'], dtype='object')
Apple DataFrame:
        Date  AAPL
0 2024-01-01   150
1 2024-01-02   155
4 2024-01-05   160
Microsoft DataFrame:
        Date  MSFT
2 2024-01-03   250
3 2024-01-04   255
Merged DataFrame shape: (5, 3)
Merged Data:
         Date  AAPL   MSFT
0 2024-01-01   150    NaN
1 2024-01-02   155    NaN
2 2024-01-03   155  250.0
3 2024-01-04   155  255.0
4 2024-01-05   160    NaN
